In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Preprocessing

In [48]:
cols = ["year", "month", "time", "NIS", "region", "prov", "munty", "x_coord", "y_coord", "crossway", "crossway_nl", "weather", "weather_nl", "road_condition", "road_condition_nl",
         "buildup", "buildup_nl", "light_condition", "light_condition_nl", "road_type", "road_type_nl", "class_accidents", "class_accidents_nl", "road_user_1", "road_user_1_nl", "road_user_2",
         "road_user_2_nl", "collision_type", "collision_type_nl", "obstacles", "obstacles_nl"]
df = pd.read_csv("../../OPENDATA_MAP_2017-2022.csv", header=0)
df = df.loc[:, ~df.columns.str.endswith('_FR')]
df.columns = cols

In [49]:
df['prov'] = df['prov'].fillna('Provincie Brussel')
df = df.dropna()

In [50]:
df.head()

,year,month,time,NIS,region,prov,munty,x_coord,y_coord,crossway,...,class_accidents,class_accidents_nl,road_user_1,road_user_1_nl,road_user_2,road_user_2_nl,collision_type,collision_type_nl,obstacles,obstacles_nl
0,2017,1,1,21015,Brussels Hoofdstedelijk Gewest,Provincie Brussel,Schaarbeek,149732.9900,172425.5100,1,...,4,met lichtgewonden,9,personenauto,8,Voetganger,5,Met een voetganger,0,Geen hindernis
1,2017,1,0,11002,Vlaams Gewest,Provincie Antwerpen,Antwerpen,152268.0690,209633.1210,1,...,4,met lichtgewonden,3,Fiets,9,personenauto,2,Tussen 2 bestuurders: Frontale botsing,0,Geen hindernis
2,2017,1,1,71016,Vlaams Gewest,Provincie Limburg,Genk,228984.9961,184597.3621,1,...,4,met lichtgewonden,9,personenauto,9,personenauto,3,Tussen 2 bestuurders: Langs achteren,0,Geen hindernis
3,2017,1,10,21001,Brussels Hoofdstedelijk Gewest,Provincie Brussel,Anderlecht,145322.2700,168391.8700,1,...,4,met lichtgewonden,9,personenauto,9,personenauto,4,Tss. 2 best.: langs opzij (voor-/achterkant-fl...,0,Geen hindernis
4,2017,1,7,11035,Vlaams Gewest,Provincie Antwerpen,Ranst,172211.1433,214752.5686,2,...,4,met lichtgewonden,9,personenauto,9,personenauto,3,Tussen 2 bestuurders: Langs achteren,0,Geen hindernis


In [51]:
for col in df.columns:
    if df[col].dtype == pd.StringDtype:
        df[col] = df[col].str.lower()
        
df = df[~df.isin(['onbekend']).any(axis=1)]
df = df[~df.isin(['andere weggebruiker']).any(axis=1)]

In [52]:
unique_list = ['light_condition_nl', 'buildup_nl', 'crossway_nl', 'road_type_nl', 'class_accidents_nl', 'road_user_1_nl', 'obstacles_nl', 'weather_nl', 'road_condition_nl', 'collision_type_nl']
for column in df[unique_list]:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}\n")

Unique values in column 'light_condition_nl': ['nacht, openbare verlichting aanwezig en ontstoken' 'dag'
 'dageraad - schemering' 'nacht, geen openbare verlichting aanwezig'
 'nacht, openb. verlicht. aanw., maar niet ontstoken']

Unique values in column 'buildup_nl': ['binnen bebouwde kom' 'buiten bebouwde kom']

Unique values in column 'crossway_nl': ['op kruispunt' 'buiten kruispunt']

Unique values in column 'road_type_nl': ['gemeenteweg' 'autosnelweg' 'gewestweg']

Unique values in column 'class_accidents_nl': ['met lichtgewonden' 'met zwaargewonden' 'met doden' 'dodelijk gewonden']

Unique values in column 'road_user_1_nl': ['personenauto' 'fiets' 'vrachtwagen' 'lichte vrachtauto' 'voetganger'
 'motorfiets' 'bromfiets' 'autobus']

Unique values in column 'obstacles_nl': ['geen hindernis' 'buiten de rijbaan: vangrails niet overgestoken'
 'buiten de rijbaan: andere paal'
 'buiten de rijbaan: vangrails overschreden' 'andere hindernis'
 'op de rijbaan: werken (signal., aarde, toestel,

In [53]:
for col in df.columns:
    if col.endswith('_nl'):
        col_prefix = col.split('_')[:-1]
        col_prefix = '_'.join(col_prefix)
        df.drop(columns=col_prefix, inplace=True)
        df.rename(columns={col: col_prefix}, inplace=True)

In [54]:
knz, gnz, zw = 'kleine niet-zwakke weggebruikers', 'grote niet-zwakke weggebruikers', 'zwakke weggebruiker'
road_user_mapping = {'personenauto': knz, 'fiets': zw, 'vrachtwagen': gnz, 'lichte vrachtauto': knz, 'voetganger': zw, 'motorfiets': zw, 'bromfiets': zw, 'autobus': gnz}
light_condition_mapping = {'nacht, openbare verlichting aanwezig en ontstoken': 'licht', 'dag': 'licht', 'dageraad - schemering': 'schemering', 'nacht, geen openbare verlichting aanwezig': 'donker', 'nacht, openb. verlicht. aanw., maar niet ontstoken': 'donker'}
weather_mapping = {'normaal': 'normaal', 'sneeuwval': 'neerslag', 'regenval': 'neerslag', 'hagelbui': 'neerslag', 'mist (zichtbaarheid minder dan 100m)': 'vermindert zicht', 'sneeuwval+hagelbui': 'neerslag', 'andere (dikke rook,...)': 'vermindert zicht', 'sterke wind, rukwind+sneeuwval': 'neerslag', 'regenval+sneeuwval': 'neerslag', 'sterke wind, rukwind': 'wind', 'regenval+sterke wind, rukwind': 'neerslag', 'regenval+mist (zichtbaarheid minder dan 100m)': 'vermindert zicht', 'regenval+hagelbui': 'neerslag', 'regenval+andere (dikke rook,...)': 'vermindert zicht'}
road_condition_mapping = {'droog': 'niet glad', 'nat, plassen': 'glad', 'ijzel, sneeuw': 'glad', 'droog+proper': 'niet glad', 'proper': 'niet glad', 'nat, plassen+ijzel of sneeuw': 'glad', 'nat, plassen+proper': 'glad', 'ijzel, sneeuw+proper': 'glad', 'nat, plassen+vuil (zand, grint, bladeren,...)': 'glad', 'vuil (zand, grint, bladeren,...)': 'niet glad', 'droog+vuil (zand, grint, bladeren,...)': 'niet glad', 'ijzel, sneeuw+vuil (zand, grint, bladeren,...)': 'glad'}

    
df['road_user_1'] = df['road_user_1'].map(road_user_mapping)
df['road_user_2'] = df['road_user_2'].map(road_user_mapping)
df['light_condition'] = df['light_condition'].map(light_condition_mapping)
df['weather'] = df['weather'].map(weather_mapping)
df['road_condition'] = df['road_condition'].map(road_condition_mapping)

In [55]:
col_types = df['collision_type'].unique()
col_type_mapping = {}
for col in col_types:
    if 'voetganger' in col or '2' in col:
        col_type_mapping[col] = '2 betrokkenen'
    elif '3' in col:
        col_type_mapping[col] = '3 betrokkenen'
    else:
        col_type_mapping[col] = '1 betrokkene'
df['collision_type'] = df['collision_type'].map(col_type_mapping)


In [56]:
obstacles = df['obstacles'].unique()
obs_mapping = {}
for obs in obstacles:
    split = obs.split()
    if split[0] == 'buiten':
        obs_mapping[obs] = 'buiten de rijbaan'
    elif split[0] == 'op':
        obs_mapping[obs] = 'op de rijbaan'
    elif split[0] == 'geen':
        obs_mapping[obs] = 'geen'
    else:
       obs_mapping[obs] = 'op de rijbaan'
df['obstacles'] = df['obstacles'].map(obs_mapping)

In [ ]:
for column in df.select_dtypes(include=['object', 'category']):
    plt.figure(figsize=(6, 4))
    df[column].value_counts().plot(kind='bar')
    plt.title(f'Distribution of {column}')
    plt.ylabel('Frequency')
    plt.xticks([])
    plt.show()

In [141]:
# removing skewed columns
df = df.drop(columns=['weather', 'class_accidents', 'obstacles'])

In [29]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

In [ ]:
ordinal_encoder = OrdinalEncoder()
onehot_encoder = OneHotEncoder()
label_encoder = LabelEncoder()

df[['light_condition', '']]